In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [2]:
dataset = pd.read_csv("C:/Users/vishn/Downloads/train.csv")
numfeat = [feature for feature in dataset.columns if dataset[feature].dtype!='O']
catfeat = [feature for feature in dataset.columns if dataset[feature].dtype=='O']
years = [feature for feature in numfeat if 'Yr' in feature or 'Year' in feature]
catna = [feature for feature in catfeat if dataset[feature].isnull().sum()>0]
nanum = [feature for feature in numfeat if dataset[feature].isnull().sum()>0]
discrete = [feature for feature in numfeat if len(dataset[feature].unique())<=25 and feature not in years]
cont = [feature for feature in numfeat if feature not in years+discrete+['Id']]

In [3]:
testdata = pd.read_csv("C:/Users/vishn/Downloads/test.csv")
tcatfeat = [feature for feature in testdata.columns if testdata[feature].dtype=='O']
tcna = [feature for feature in tcatfeat if testdata[feature].isnull().sum()>0]
tnumfeat = [feature for feature in testdata.columns if testdata[feature].dtype!='O']
tnna = [feature for feature in tnumfeat if testdata[feature].isnull().sum()>0]
tyears = [feature for feature in tnumfeat if 'Yr' in feature or 'Year' in feature]
tdiscrete = [feature for feature in tnumfeat if len(testdata[feature].unique())<=25 and feature not in tyears]
for feature in tcatfeat:
    if feature not in catfeat:
        print(feature)

In [4]:
for feature in catna:
    dataset[feature] = dataset[feature].fillna("Missing")

for feature in nanum:
    med = dataset[feature].median()
    dataset[feature+"_na"] = np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(med, inplace=True)

for feature in years:
    if feature != "YrSold":
        dataset[feature] = dataset["YrSold"]-dataset[feature]

In [5]:
for feature in tcna:
    testdata[feature] = testdata[feature].fillna("Missing")

for feature in tnna:
    medv = testdata[feature].median()
    if feature in nanum:
        testdata[feature+"_na"] = np.where(testdata[feature].isnull(),1,0)
    testdata[feature].fillna(medv,inplace = True)

for feature in tyears:
    if feature != "YrSold":
        testdata[feature] = testdata['YrSold']-testdata[feature]

In [6]:
for feature in catfeat:
    temp = dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df = temp[temp>0.01].index
    dataset[feature] = np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare')
    testdata[feature] = np.where(testdata[feature].isin(temp_df),testdata[feature],'Rare')
    labels_ordered = dataset.groupby(feature)['SalePrice'].mean().sort_values().index
    labels__ordered = {k:i for i,k in enumerate(labels_ordered,1)}
    dataset[feature] = dataset[feature].map(labels__ordered)
    testdata[feature] = testdata[feature].map(labels__ordered)

In [7]:
for feature in numfeat:
    if 0 in dataset[feature].unique():
        pass
    else:
        dataset[feature] = np.log(dataset[feature])

for feature in tnumfeat:
    if 0 in testdata[feature].unique():
        pass
    else:
        testdata[feature] = np.log(testdata[feature])

In [8]:
testdata.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,7.286876,2.995732,3,4.382027,9.360655,2,3,1,2,2,...,3,3,0,1.791759,7.60589,3,4,0,0,0
1,7.287561,2.995732,4,4.394449,9.565704,2,3,2,2,2,...,5,2,12500,1.791759,7.60589,3,4,0,0,0
2,7.288244,4.094345,4,4.304065,9.534595,2,3,2,2,2,...,3,3,0,1.098612,7.60589,3,4,0,0,0
3,7.288928,4.094345,4,4.356709,9.208138,2,3,2,2,2,...,5,3,0,1.791759,7.60589,3,4,0,0,0
4,7.289611,4.787492,4,3.761200,8.518193,2,3,2,4,2,...,5,3,0,0.000000,7.60589,3,4,0,0,0
5,7.290293,4.094345,4,4.317488,9.210340,2,3,2,2,2,...,5,3,0,1.386294,7.60589,3,4,0,0,0
6,7.290975,2.995732,4,4.204693,8.984694,2,3,2,2,2,...,4,1,500,1.098612,7.60589,3,4,1,0,0
7,7.291656,4.094345,4,4.143135,9.036225,2,3,2,2,2,...,5,3,0,1.609438,7.60589,3,4,0,0,0
8,7.292337,2.995732,4,4.442651,9.227787,2,3,1,2,2,...,5,3,0,0.693147,7.60589,3,4,0,0,0
9,7.293018,2.995732,4,4.248495,9.035987,2,3,1,2,2,...,3,3,0,1.386294,7.60589,3,4,0,0,0


In [9]:
from sklearn.preprocessing import MinMaxScaler
feature_scale = [feature for feature in dataset.columns if feature not in ['Id','SalePrice']]
scaler = MinMaxScaler()
scaler.fit_transform(dataset[feature_scale])
scaler.transform(testdata[feature_scale])

array([[0.        , 0.5       , 0.49506375, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.49966182, ..., 0.        , 0.        ,
        0.        ],
       [0.48799196, 0.75      , 0.46620707, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.75      , 0.75162522, ..., 0.        , 0.        ,
        0.        ],
       [0.6427061 , 0.75      , 0.40071794, ..., 0.        , 0.        ,
        1.        ],
       [0.48799196, 0.75      , 0.46620707, ..., 0.        , 0.        ,
        0.        ]])

In [13]:
X_train = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.fit_transform(dataset[feature_scale]), columns=feature_scale)],
                    axis=1)
X_test = pd.concat([testdata[['Id']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(testdata[feature_scale]), columns=feature_scale)],
                    axis=1)
#X_train.to_csv('HP_train.csv',index = False)
#X_test.to_csv('HP_test.csv',index = False)

In [16]:
X_train.head(10)

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,0.000000,12.247694,0.487992,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,...,1.0,1.0,0.000000,0.278943,0.500249,0.666667,0.75,0.0,0.0,0.0
1,0.693147,12.109011,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,...,1.0,1.0,0.000000,0.647685,0.250187,0.666667,0.75,0.0,0.0,0.0
2,1.098612,12.317167,0.487992,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,...,1.0,1.0,0.000000,0.884228,0.500249,0.666667,0.75,0.0,0.0,0.0
3,1.386294,11.849398,0.556464,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,...,1.0,1.0,0.000000,0.278943,0.000000,0.666667,0.00,0.0,0.0,0.0
4,1.609438,12.429216,0.487992,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,...,1.0,1.0,0.000000,1.000000,0.500249,0.666667,0.75,0.0,0.0,0.0
5,1.791759,11.870600,0.407007,0.75,0.517503,0.466761,1.0,1.0,0.333333,0.333333,...,0.5,0.0,0.045161,0.926628,0.750187,0.666667,0.75,0.0,0.0,0.0
6,1.945910,12.634603,0.000000,0.75,0.471175,0.400943,1.0,1.0,0.000000,0.333333,...,1.0,1.0,0.000000,0.836829,0.250187,0.666667,0.75,0.0,0.0,0.0
7,2.079442,12.206073,0.487992,0.75,0.440313,0.406643,1.0,1.0,0.333333,0.333333,...,1.0,0.0,0.022581,0.964984,0.750187,0.666667,0.75,1.0,0.0,0.0
8,2.197225,11.774520,0.407007,0.25,0.328426,0.303205,1.0,1.0,0.000000,0.333333,...,1.0,1.0,0.000000,0.557886,0.500249,0.666667,0.00,0.0,0.0,0.0
9,2.302585,11.678440,1.000000,0.75,0.321097,0.340903,1.0,1.0,0.000000,0.333333,...,1.0,1.0,0.000000,0.000000,0.500249,0.666667,0.75,0.0,0.0,0.0


In [17]:
y_train = X_train['SalePrice']
X_train.drop(['Id','SalePrice'],axis=1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,0.487992,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,...,1.00,1.0,0.00000,0.278943,0.500249,0.666667,0.75,0.0,0.0,0.0
1,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.25,...,1.00,1.0,0.00000,0.647685,0.250187,0.666667,0.75,0.0,0.0,0.0
2,0.487992,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,...,1.00,1.0,0.00000,0.884228,0.500249,0.666667,0.75,0.0,0.0,0.0
3,0.556464,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.50,...,1.00,1.0,0.00000,0.278943,0.000000,0.666667,0.00,0.0,0.0,0.0
4,0.487992,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,1.0,0.25,...,1.00,1.0,0.00000,1.000000,0.500249,0.666667,0.75,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.487992,0.75,0.400718,0.353592,1.0,1.0,0.000000,0.333333,1.0,0.00,...,1.00,1.0,0.00000,0.836829,0.250187,0.666667,0.75,0.0,0.0,0.0
1456,0.000000,0.75,0.517503,0.453273,1.0,1.0,0.000000,0.333333,1.0,0.00,...,0.50,1.0,0.00000,0.278943,1.000000,0.666667,0.75,0.0,0.0,0.0
1457,0.556464,0.75,0.423859,0.379597,1.0,1.0,0.000000,0.333333,1.0,0.00,...,0.75,0.0,0.16129,0.647685,1.000000,0.666667,0.75,0.0,0.0,0.0
1458,0.000000,0.75,0.434909,0.393688,1.0,1.0,0.000000,0.333333,1.0,0.00,...,1.00,1.0,0.00000,0.557886,1.000000,0.666667,0.75,0.0,0.0,0.0


## Feature Selection


In [23]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [27]:
featmodel = SelectFromModel(Lasso(alpha = 0.005, random_state=0))
featmodel.fit(X_train,y_train)
selectfeat = X_train.columns[(featmodel.get_support())]
X_train = X_train[selectfeat]